In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [2]:
# === Configuration ===
DATA_DIR = 'isl'
images_per_class = 1200

# === Load only valid class folders ===
LABELS = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])
LABEL_TO_INDEX = {label: idx for idx, label in enumerate(LABELS)}
NUM_CLASSES = len(LABELS)

# === MediaPipe Setup ===
mp_hands = mp.solutions.hands

In [3]:
def extract_landmarks(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    with mp_hands.Hands(static_image_mode=True) as hands:
        result = hands.process(image_rgb)
        if result.multi_hand_landmarks:
            landmarks = []
            for lm in result.multi_hand_landmarks[0].landmark:
                landmarks.extend([lm.x, lm.y, lm.z])
            return landmarks
    return None

In [4]:
def load_dataset():
    X, y = [], []
    for label in LABELS:
        label_dir = os.path.join(DATA_DIR, label)
        count = 0
        for img_name in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_name)
            try:
                landmarks = extract_landmarks(img_path)
                if landmarks:
                    X.append(landmarks)
                    y.append(LABEL_TO_INDEX[label])
                    count += 1
                if count >= images_per_class:
                    break
            except Exception as e:
                print(f"Error in {img_path}: {e}")
    return np.array(X), to_categorical(y, NUM_CLASSES)

In [5]:
# === Load and split data ===
print("Loading data...")
X, y = load_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Loading data...


In [10]:
# === Compute class weights ===
y_int = np.argmax(y_train, axis=1)
class_weights = compute_class_weight('balanced', classes=np.unique(y_int), y=y_int)
class_weights_dict = dict(enumerate(class_weights))

In [11]:
# Define model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax')
])

c:\users\karan\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("Training model...")
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), class_weight=class_weights_dict)

Training model...
Epoch 1/50
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3477 - loss: 2.4035 - val_accuracy: 0.9156 - val_loss: 0.4055
Epoch 2/50
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8445 - loss: 0.5604 - val_accuracy: 0.9825 - val_loss: 0.1355
Epoch 3/50
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9229 - loss: 0.2861 - val_accuracy: 0.9870 - val_loss: 0.0690
Epoch 4/50
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9484 - loss: 0.1936 - val_accuracy: 0.9914 - val_loss: 0.0474
Epoch 5/50
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9623 - loss: 0.1452 - val_accuracy: 0.9905 - val_loss: 0.0344
Epoch 6/50
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9703 - loss: 0.1130 - val_accuracy: 0.9942 - val_loss: 0.0248
Epoch 7/50
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9727 - loss: 0.0973 - val_accuracy: 0.9963 - val_loss: 0.0193
Epoch 8/50
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9772

In [13]:
# Save model
model.save('model/isl_cnn_model.h5')
print("✅ Model trained and saved successfully.")

✅ Model trained and saved successfully.
